In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import models.conv_net as cnet

In [2]:
cnet.ConvNet(cnet.DEFAULT_PARAMS)

Layer 1,	type: conv2d,	filter: [4, 4],	FOV: 4
Layer 2,	type: pool,	filter: [2, 2],	FOV: 5
Layer 3,	type: conv2d,	filter: [5, 5],	FOV: 13
Layer 4,	type: pool,	filter: [2, 2],	FOV: 15
Layer 5,	type: conv2d,	filter: [4, 4],	FOV: 27
Layer 6,	type: pool,	filter: [2, 2],	FOV: 31
Layer 7,	type: conv2d,	filter: [4, 4],	FOV: 55
Layer 8,	type: pool,	filter: [2, 2],	FOV: 63
Layer 9,	type: conv2d,	filter: [3, 3],	FOV: 95
Layer 10,	type: conv2d,	filter: [1, 1],	FOV: 95


<models.conv_net.ConvNet instance at 0x7fdf246473b0>

In [7]:
cnet.DEFAULT_PARAMS

{'fov': 95,
 'input': 195,
 'layers': [{'activation_fn': <function tensorflow.python.ops.gen_nn_ops.relu>,
   'filter_size': 4,
   'n_feature_maps': 48,
   'type': 'conv2d'},
  {'filter_size': 2, 'type': 'pool'},
  {'activation_fn': <function tensorflow.python.ops.gen_nn_ops.relu>,
   'filter_size': 5,
   'n_feature_maps': 68,
   'type': 'conv2d'},
  {'filter_size': 2, 'type': 'pool'},
  {'activation_fn': <function tensorflow.python.ops.gen_nn_ops.relu>,
   'filter_size': 4,
   'n_feature_maps': 100,
   'type': 'conv2d'},
  {'filter_size': 2, 'type': 'pool'},
  {'activation_fn': <function tensorflow.python.ops.gen_nn_ops.relu>,
   'filter_size': 4,
   'n_feature_maps': 128,
   'type': 'conv2d'},
  {'filter_size': 2, 'type': 'pool'},
  {'activation_fn': <function tensorflow.python.ops.gen_nn_ops.relu>,
   'filter_size': 3,
   'n_feature_maps': 200,
   'type': 'conv2d'},
  {'activation_fn': <function models.conv_net.<lambda>>,
   'filter_size': 1,
   'n_feature_maps': 2,
   'type': 'conv

In [20]:
DEEPER_PARAMS = {
    'model_name': 'N4_widened',
    'fov': 107,  # Sanity check
    'input': 195,
    'output': 101,
    'learning_rate': 0.0001,
    'layers': [
        {'type': 'conv2d', 'filter_size': 4, 'n_feature_maps': 48, 'activation_fn': tf.nn.relu},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 5, 'n_feature_maps': 68, 'activation_fn': tf.nn.relu},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 4, 'n_feature_maps': 100, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 4, 'n_feature_maps': 200, 'activation_fn': tf.nn.relu},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 4, 'n_feature_maps': 96, 'activation_fn': tf.nn.relu},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 200, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 1, 'n_feature_maps': 2, 'activation_fn': lambda x: x},
    ]
}

In [23]:
VD2D = {
    'model_name': 'VD2D',
    'fov': 109,  # Sanity check
    'input': 209,
    'output': 101,
    'learning_rate': 0.001,
    'layers': [
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 24, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 24, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 2, 'n_feature_maps': 24, 'activation_fn': tf.nn.tanh},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 36, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 36, 'activation_fn': tf.nn.tanh},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 48, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 48, 'activation_fn': tf.nn.tanh},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 60, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 60, 'activation_fn': tf.nn.tanh},
        {'type': 'pool', 'filter_size': 2},
        {'type': 'conv2d', 'filter_size': 3, 'n_feature_maps': 200, 'activation_fn': tf.nn.relu},
        {'type': 'conv2d', 'filter_size': 1, 'n_feature_maps': 2, 'activation_fn': lambda x: x},
    ]
    
}

In [24]:
cnet.ConvNet(VD2D)

Layer 1,	type: conv2d,	filter: [3, 3],	FOV: 3
Layer 2,	type: conv2d,	filter: [3, 3],	FOV: 5
Layer 3,	type: conv2d,	filter: [2, 2],	FOV: 6
Layer 4,	type: pool,	filter: [2, 2],	FOV: 7
Layer 5,	type: conv2d,	filter: [3, 3],	FOV: 11
Layer 6,	type: conv2d,	filter: [3, 3],	FOV: 15
Layer 7,	type: pool,	filter: [2, 2],	FOV: 17
Layer 8,	type: conv2d,	filter: [3, 3],	FOV: 25
Layer 9,	type: conv2d,	filter: [3, 3],	FOV: 33
Layer 10,	type: pool,	filter: [2, 2],	FOV: 37
Layer 11,	type: conv2d,	filter: [3, 3],	FOV: 53
Layer 12,	type: conv2d,	filter: [3, 3],	FOV: 69
Layer 13,	type: pool,	filter: [2, 2],	FOV: 77
Layer 14,	type: conv2d,	filter: [3, 3],	FOV: 109
Layer 15,	type: conv2d,	filter: [1, 1],	FOV: 109


AssertionError: 